## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 13
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_13


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 49.7 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Kansas City Chiefs,12,2022,29.2,80.0,8.1,306.5,104.2,12.4,22.5,...,10.5,106.2,20.750000,11.666667,12.750000,0.058091,2.500000,0.583333,3.428571,3.750000
1,Buffalo Bills,12,2022,27.8,50.0,7.6,274.5,95.0,11.9,17.4,...,10.4,101.8,17.583333,11.916667,12.666667,0.098765,2.083333,0.666667,1.875000,2.272727


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[12, 13]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_13,0.399510,0.333554,0.211172,0.319178,0.404348,0.333553,0.403743
1,Buffalo Bills,2022,week_13,0.496466,0.129181,0.084638,0.186587,0.361080,0.251590,0.378003
2,Philadelphia Eagles,2022,week_13,0.388815,0.265670,0.145037,0.264028,0.356409,0.283992,0.360460


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_13,0.399510,0.333554,0.211172,0.319178,0.404348,0.333553,0.403743
1,Buffalo Bills,2022,week_13,0.496466,0.129181,0.084638,0.186587,0.361080,0.251590,0.378003
2,Philadelphia Eagles,2022,week_13,0.388815,0.265670,0.145037,0.264028,0.356409,0.283992,0.360460
3,Miami Dolphins,2022,week_13,0.181920,0.243447,0.145639,0.281981,0.369916,0.244581,0.346416
10,Dallas Cowboys,2022,week_13,0.268995,0.105188,0.078778,0.196052,0.346569,0.199117,0.336872
9,San Francisco 49ers,2022,week_13,0.172163,0.225996,0.099363,0.264652,0.349958,0.222426,0.327734
4,Cincinnati Bengals,2022,week_13,0.040744,0.125021,0.072013,0.228517,0.362456,0.165750,0.322242
8,Seattle Seahawks,2022,week_13,0.013861,0.140322,0.075568,0.205409,0.356996,0.158431,0.314104
16,Tampa Bay Buccaneers,2022,week_13,0.020555,0.072168,0.073450,0.199974,0.345818,0.142393,0.305160
28,Tennessee Titans,2022,week_13,0.007784,0.010955,0.049657,0.191399,0.340966,0.120152,0.299319
